## Download data from API

In [19]:
import requests
import pandas as pd
import time
from datetime import datetime

### Download API token from file

In [21]:
def get_api_token_from_file(filename):
    try:
        with open(filename, 'r') as file:
            api_token = file.read().strip()
        return api_token
    except FileNotFoundError:
        print(f"Plik {filename} nie został znaleziony.")
        return None
    except Exception as e:
        print(f"Wystąpił błąd podczas odczytu pliku {filename}: {str(e)}")
        return None
    
api_token_file = 'd:\\praca_inzynierska\\Pliki_projektu\\api_token.txt'
my_api_token = get_api_token_from_file(api_token_file)
if my_api_token:
    print(f"Pobrano API token z pliku")
else:
    print("Nie udało się pobrać API token.")

Pobrano API token z pliku


### Get player attribute data from API and create CSV file

In [22]:
headers = {
    'X-Auth-Token': my_api_token
}

def get_data_from_api(url):
    while True:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = int(response.json().get('message').split()[-2])
            print(f"Limit żądań przekroczony. Czekanie {wait_time} sekund.")
            time.sleep(wait_time)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

def get_all_teams():
    url = 'https://api.football-data.org/v2/teams'
    data = get_data_from_api(url)
    if data:
        teams = data['teams']
        return pd.DataFrame(teams)
    return pd.DataFrame()

def get_players_for_team(team_id):
    url = f'https://api.football-data.org/v2/teams/{team_id}'
    data = get_data_from_api(url)
    if data and 'squad' in data:
        players = data['squad']
        return pd.DataFrame(players)
    return pd.DataFrame()

def get_player_attributes_from_2022(player_id):
    url = f'https://api.football-data.org/v2/players/{player_id}'
    data = get_data_from_api(url)
    if data:
        last_update = datetime.strptime(data['lastUpdated'], '%Y-%m-%dT%H:%M:%SZ')
        if last_update.year >= 2022:
            return data
    return None

def collect_data_from_2022():
    teams = get_all_teams()
    all_players = []
    
    for _, team in teams.iterrows():
        team_id = team['id']
        players = get_players_for_team(team_id)
        if not players.empty:
            players['team_id'] = team_id
            players['team_name'] = team['name']
            players['league_id'] = team['area']['id']
            players['league_name'] = team['area']['name']
            all_players.append(players)
        time.sleep(1)
    
    all_players_cleaned = [df for df in all_players if not df.empty]
    
    if all(all_players_cleaned[0].columns.tolist() == df.columns.tolist() for df in all_players_cleaned):
        all_players_df = pd.concat(all_players_cleaned, ignore_index=True)
    else:
        print("Nie można połączyć DataFrame'ów: różne kolumny lub typy danych.")
        return pd.DataFrame()

    all_player_attributes = []
    for _, player in all_players_df.iterrows():
        player_id = player['id']
        try:
            player_id = int(player_id)
        except ValueError:
            print(f"Błąd: ID zawodnika '{player_id}' nie jest liczbą całkowitą.")
            continue
        
        player_attributes = get_player_attributes_from_2022(player_id)
        if player_attributes:
            player_attributes['player_id'] = player_id
            player_attributes['team_id'] = player['team_id']
            player_attributes['team_name'] = player['team_name']
            player_attributes['league_id'] = player['league_id']
            player_attributes['league_name'] = player['league_name']
            all_player_attributes.append(player_attributes)
        time.sleep(1)
    
    all_player_attributes_df = pd.DataFrame(all_player_attributes)
    return all_player_attributes_df

def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"Dane zostały zapisane do pliku {filename}")

def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"Dane zostały zapisane do pliku {filename}")


In [ ]:
data = collect_data_from_2022()
if not data.empty:
    save_to_csv(data, 'd:\\praca_inzynierska\\Pliki_projektu\\player_attributes.csv')